In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd
from collections import OrderedDict
from scipy.interpolate import interp1d

In [2]:
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
import lsst.afw.cameraGeom.utils as cgUtils
from lsst.daf.persistence import Butler, NoResults

In [26]:
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Bandpass, LSSTdefaults
from lsst.sims.utils import angularSeparation

In [4]:
#rname = 'R10'
#rname = 'R22'
#rname = 'R01'
#rname = 'R11'
#rname = 'R20'
#rname = 'R21'
#rname = 'R30'
#rname = 'R12'
#rname = 'R02'
#rname = 'R31'
#rname = 'R03'
#rname = 'R34'
#rname = 'R13'
#rname = 'R23'
#rname = 'R14'
#rname = 'R32'
#rname = 'R24'
#rname = 'R41'
rname = 'R42'
#rname = 'R33'
#rname = 'R43'

In [5]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
useDefault = False
try:
    if np.isnan(dd.rtm[rname]):
        useDefault = True
        print('no data yet, use default')
except TypeError:
    pass

### Default photometric parameters, as used in standard m5 calculations
Note that effarea is not in this list here, because it varies with field.

The read noise is not in this list either, because it varies by amp.

In [6]:
exptime=15 
nexp=2
othernoise=0 
darkcurrent=0.2
X=1.0

In [41]:
#we do not need this cell to calculate m5, but these FWHMeff are the default values we use actually
lsstDefaults = LSSTdefaults()
for f in filters:
    print(lsstDefaults.FWHMeff(f), lsstDefaults.m5(f))

0.92 23.68
0.76 22.6
0.87 24.89
0.83 24.43
0.78 24.45
0.8 24.0


### Set up throughputs for hardware and atmosphere. Use the default detector QE as in syseng_throughput for now

In [7]:
# Add losses to each component?
addLosses = True
defaultDirs = st.setDefaultDirs()
defaultDirs

{'detector': '/home/bxin/notebooks/syseng_throughputs/components/camera/detector/joint_minimum',
 'lens1': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens1',
 'lens2': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens2',
 'lens3': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens3',
 'filters': '/home/bxin/notebooks/syseng_throughputs/components/camera/filters',
 'mirror1': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror1',
 'mirror2': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror2',
 'mirror3': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror3',
 'atmosphere': '/home/bxin/notebooks/syseng_throughputs/siteProperties'}

In [8]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
mirror1 = st.buildMirror(defaultDirs['mirror1'], addLosses)
mirror2 = st.buildMirror(defaultDirs['mirror2'], addLosses)
mirror3 = st.buildMirror(defaultDirs['mirror3'], addLosses)
lens1 = st.buildLens(defaultDirs['lens1'], addLosses)
lens2 = st.buildLens(defaultDirs['lens2'], addLosses)
lens3 = st.buildLens(defaultDirs['lens3'], addLosses)
filters = st.buildFilters(defaultDirs['filters'], addLosses)

vendor = dd.vendor[rname]
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector0 = st.buildDetector(vendorDir, addLosses) #design QE from this vendor
detector = Bandpass()

### Butler access to the QE data

In [9]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

#This is no longer needed after tickets/DM-22605
#from lsst.obs.lsst.lsstCamMapper import LsstCamMapper
#mapper = LsstCamMapper()
#lsstcam = mapper.camera

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Prepare vignetting function

In [10]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframes for this raft

In [11]:
filterlist = tuple([s for s in filters])
alist = ('raDeg', 'decDeg', 'radDeg', 'effarea', 'readnoise')
detectors = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    detectors.append(det.getName())
adf = pd.DataFrame(index=alist, columns=detectors, dtype=object)
m5df = pd.DataFrame(index=filterlist, columns=detectors, dtype=object)

In [12]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    key = rname+'_'+dname
    raDeg = {}
    decDeg = {}
    readnoise = {}
    for amp in det:
        i = amp.getName()
        amp_point = amp.getBBox().getCenter()
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [raDeg[i], decDeg[i]] = np.degrees(raDec)
        #print(key, i, raDec)
        if useDefault:
            readnoise[i] = 8.8
        else:
            readnoise[i] = amp.getReadNoise()
    adf[key].loc['raDeg'] = list(OrderedDict(sorted(raDeg.items())).values())
    adf[key].loc['decDeg'] = list(OrderedDict(sorted(decDeg.items())).values())
    adf[key].loc['readnoise'] = list(OrderedDict(sorted(readnoise.items())).values())
        
    #effetive area
    radius = angularSeparation(0., 0., adf[key]['raDeg'], adf[key]['decDeg'])
    adf[key].loc['radDeg'] = list(radius)
    adf[key].loc['effarea'] = list(np.interp(radius, vr, vv)*np.pi*(M1D/2)**2)

In [13]:
adf

,R42_S00,R42_S01,R42_S02,R42_S10,R42_S11,R42_S12,R42_S20,R42_S21,R42_S22
raDeg,"[-0.3310833333333334, -0.3028055555555556, -0....","[-0.09619444444444444, -0.06791666666666668, -...","[0.1386388888888889, 0.16691666666666666, 0.19...","[-0.33080555555555563, -0.3025277777777778, -0...","[-0.09569444444444446, -0.06741666666666668, -...","[0.13919444444444445, 0.16747222222222222, 0.1...","[-0.33008333333333334, -0.30180555555555555, -...","[-0.0951388888888889, -0.06686111111111114, -0...","[0.1396388888888889, 0.16791666666666666, 0.19..."
decDeg,"[1.1216666666666668, 1.1216666666666668, 1.121...","[1.1209444444444443, 1.1209444444444443, 1.120...","[1.1206666666666667, 1.1206666666666667, 1.120...","[1.3565, 1.3565, 1.3564999999999998, 1.3564999...","[1.3560555555555556, 1.3560555555555553, 1.356...","[1.3555000000000001, 1.3555000000000001, 1.355...","[1.5913888888888887, 1.5913888888888887, 1.591...","[1.5909444444444445, 1.5909444444444445, 1.590...","[1.5905000000000002, 1.5905000000000002, 1.590..."
radDeg,"[1.1695034334539784, 1.1618156487621303, 1.154...","[1.125063837320171, 1.1229997919038877, 1.1216...","[1.1292086025850416, 1.1330275364585176, 1.137...","[1.396246438953936, 1.3898194866103237, 1.3839...","[1.3594272273723056, 1.357730027666303, 1.3566...","[1.3626267752007601, 1.365804511052421, 1.3695...","[1.625252467577429, 1.6197474386149016, 1.6147...","[1.593785841040167, 1.5923484180624896, 1.5914...","[1.5966165038801796, 1.5993370545539594, 1.602..."
effarea,"[33.561792066044895, 33.5752324936103, 33.5912...","[33.648290093666446, 33.65117352406033, 33.653...","[33.6424999390284, 33.63716496436353, 33.63086...","[32.92554590494814, 32.94577800379144, 32.9642...","[33.076821191000846, 33.08335645384732, 33.087...","[33.061754206231505, 33.04619508429417, 33.027...","[31.75813063724522, 31.782116587068323, 31.820...","[31.971100407129416, 31.979649419289405, 31.98...","[31.954265160012937, 31.93808480033231, 31.914..."
readnoise,"[6.72664, 6.86398, 6.80134, 6.80359, 6.89268, ...","[6.3264, 6.15465, 6.19607, 6.21983, 6.29398, 6...","[12.4259, 13.4357, 14.2171, 14.4081, 13.8188, ...","[6.34781, 6.38036, 6.47556, 6.41984, 6.57553, ...","[6.35582, 6.4962, 6.66836, 6.64662, 6.66106, 6...","[6.70244, 6.90421, 7.0441, 7.1694, 7.18389, 7....","[8.60859, 8.86458, 8.93421, 8.89687, 8.89008, ...","[6.33231, 6.52467, 6.27177, 6.38031, 6.38624, ...","[6.33007, 6.40012, 6.39609, 6.37229, 6.51182, ..."


In [14]:
ampList = list(OrderedDict(sorted(raDeg.items())).keys())

In [15]:
# this is needed if there are only 6 QE measurements per amp
idx = np.where(detector0.sb>0.01)
idx1=idx[0][0]-1
idx2=idx[0][-1]+1

x1 = detector0.wavelen[idx1]
y1 = detector0.sb[idx1]
x2 = detector0.wavelen[idx2]
y2 = detector0.sb[idx2]

In [16]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
        
    vendor = det.getSerial()[:3].lower()
    assert useDefault or dd.vendor[rname].lower() == vendor
    vendorDir = defaultDirs['detector']+'/../'+vendor
    print('Calculating m5 for %s_%s'%(rname,dname))
    
    key = rname+'_'+dname
    for f in filters:
        m5df[key][f] = [-1.]*len(ampList)
    
    for amp in det:
        
        try:
            qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, taiObs='2000-01-01T00:00:00')
            wavelen = detector0.wavelen 

            k = amp.getName()
            if len(qe_curve.data[k][0])>10:
                wlen = qe_curve.data[k][0]
                eff = qe_curve.data[k][1]
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='quadratic')
            else:
                aa = np.append(x1, qe_curve.data[k][0].value)
                aa = np.append(aa, x2)
                wlen = aa * qe_curve.data[k][0].unit

                aa = np.append(y1, qe_curve.data[k][1].value)
                aa = np.append(aa, y2)
                eff = aa * qe_curve.data[k][1].unit
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='slinear')#quadratic causes overshoot
            
            sb = f(wavelen)*0.01
            #alternatively we could do (only for >10 QE measurements)
            #amp_point = amp.getBBox().getCenter()
            #sb = qe_curve.evaluate(det, amp_point, wavelen* u.nm, kind='quadratic').value*.01 #unit was percent in CALIB data

            sb[np.isnan(sb)] = 0
            if np.max(sb)>1.5:
                print('These seem too LARGE ', k)
                print(np.max(sb))
                sb = 0
            if np.max(sb)<0.01:
                print('dead channel: %s, max sb = %.2f'%(key, np.max(sb)))
                continue;
                
            detector.setBandpass(wavelen, sb)
                
            #detector losses  
            #os.listdir(vendorDir)
            detLosses = Bandpass()
            detLosses.readThroughput(os.path.join(vendorDir, '%s_Losses/det_Losses.dat' % (vendor)))
                
            #build hardware and system
            hardware = {}
            system = {}
            for f in filters:
                sb = mirror1.sb * mirror2.sb *mirror3.sb
                sb *= lens1.sb * lens2.sb * lens3.sb * filters[f].sb
                sb *= detector.sb * detLosses.sb
                
                hardware[f] = Bandpass()
                hardware[f].setBandpass(wavelen, sb)
                system[f] = Bandpass()
                system[f].setBandpass(wavelen, sb * atmos.sb)
                
        except NoResults:
            if not useDefault:
                print('No results found for this detector')
            assert useDefault
            
            hardware, system = st.buildHardwareAndSystem(defaultDirs)
            
        #calculate m5      
        iamp = ampList.index(amp.getName())
        effarea = adf[key]['effarea'][iamp]*100**2 #convert to cm^2
        readnoise = adf[key]['readnoise'][iamp]
        
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=15, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        for f in filters:
            m5df[key][f][iamp] = m5.m5[f]


Calculating m5 for R42_S00
Calculating m5 for R42_S01
Calculating m5 for R42_S02
Calculating m5 for R42_S10
Calculating m5 for R42_S11
Calculating m5 for R42_S12
Calculating m5 for R42_S20
Calculating m5 for R42_S21
Calculating m5 for R42_S22


In [17]:
m5df

,R42_S00,R42_S01,R42_S02,R42_S10,R42_S11,R42_S12,R42_S20,R42_S21,R42_S22
u,"[24.09667288856489, 24.093538486420435, 24.100...","[24.02860288058413, 24.040422757633507, 24.032...","[23.682922308051857, 23.613879040542958, 23.56...","[24.073931807698383, 24.076227993865437, 24.07...","[24.044352394700027, 24.03037848605102, 24.013...","[23.911180747292196, 23.89019563440337, 23.872...","[23.858592518647775, 23.839412007671832, 23.83...","[23.957206679141144, 23.937098384697485, 23.95...","[23.964879802480617, 23.959780090309295, 23.95..."
y,"[22.60038470912795, 22.601690090090226, 22.603...","[22.6219434918153, 22.6234741456457, 22.623360...","[22.56871316320479, 22.561617845582084, 22.555...","[22.61191485706301, 22.613589710723623, 22.614...","[22.611538030632126, 22.608755083654227, 22.60...","[22.61593351500592, 22.617183160492292, 22.617...","[22.5734277170876, 22.57520618678376, 22.57681...","[22.584477035629313, 22.587107971351053, 22.58...","[22.61935855974503, 22.61944643708501, 22.6184..."
g,"[24.902322093826207, 24.90166707051041, 24.906...","[24.922049516109002, 24.928580848888924, 24.92...","[24.732587394197672, 24.694501099253635, 24.66...","[24.906336981201207, 24.905931574804896, 24.90...","[24.915648292676675, 24.912165171248557, 24.90...","[24.902184313126142, 24.898278371430337, 24.89...","[24.812003539913448, 24.802200363200416, 24.80...","[24.901888303399804, 24.89890368253106, 24.904...","[24.86583350885836, 24.86567196969078, 24.8645..."
r,"[24.43155524188687, 24.432513574066157, 24.434...","[24.430729513067078, 24.43440569744672, 24.433...","[24.330618179287843, 24.305992195448106, 24.28...","[24.42966562564647, 24.431281856758936, 24.430...","[24.434452337482718, 24.431635605848808, 24.42...","[24.408573149233483, 24.405907737855117, 24.40...","[24.368251508561034, 24.365034855809764, 24.36...","[24.403705142307317, 24.40169054142002, 24.404...","[24.399903991103624, 24.399046879789474, 24.39..."
z,"[23.434288547249785, 23.43645875130191, 23.438...","[23.445412485962517, 23.448012995364884, 23.44...","[23.386655264938586, 23.375365901584388, 23.36...","[23.42921238018625, 23.431638893404887, 23.431...","[23.435182507479006, 23.433696996735744, 23.43...","[23.431815601791115, 23.43148900596424, 23.431...","[23.394235703006192, 23.395248371254624, 23.39...","[23.42155447902566, 23.4210939588976, 23.42217...","[23.431425447472307, 23.431922549260662, 23.43..."
i,"[23.998908851675267, 24.00072978552631, 24.002...","[24.0031699564405, 24.00607971935881, 24.00520...","[23.92554293194894, 23.908385832625875, 23.894...","[23.994455308092228, 23.99702177915137, 23.996...","[24.000133941639394, 23.99789665506237, 23.996...","[23.982578323030403, 23.98043484606718, 23.978...","[23.947253232628515, 23.94679018065851, 23.947...","[23.975587585926128, 23.97405332555403, 23.975...","[23.98133267749364, 23.981023760131556, 23.979..."


In [18]:
dfDir = os.path.join('m5_output', rname)

In [19]:
if not os.path.exists(dfDir):
    os.mkdir(dfDir)
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
adf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
m5df.to_csv(dfPath)

### Make sure we can read out the dataframes correctly

In [20]:
from ast import literal_eval

In [21]:
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [22]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

raDeg        [-0.3310833333333334, -0.3028055555555556, -0....
decDeg       [1.1216666666666668, 1.1216666666666668, 1.121...
radDeg       [1.1695034334539784, 1.1618156487621303, 1.154...
effarea      [33.561792066044895, 33.5752324936103, 33.5912...
readnoise    [6.72664, 6.86398, 6.80134, 6.80359, 6.89268, ...
Name: R42_S00, dtype: object

In [23]:
adf

,R42_S00,R42_S01,R42_S02,R42_S10,R42_S11,R42_S12,R42_S20,R42_S21,R42_S22
raDeg,"[-0.3310833333333334, -0.3028055555555556, -0....","[-0.09619444444444444, -0.06791666666666668, -...","[0.1386388888888889, 0.16691666666666666, 0.19...","[-0.33080555555555563, -0.3025277777777778, -0...","[-0.09569444444444446, -0.06741666666666668, -...","[0.13919444444444445, 0.16747222222222222, 0.1...","[-0.33008333333333334, -0.30180555555555555, -...","[-0.0951388888888889, -0.06686111111111114, -0...","[0.1396388888888889, 0.16791666666666666, 0.19..."
decDeg,"[1.1216666666666668, 1.1216666666666668, 1.121...","[1.1209444444444443, 1.1209444444444443, 1.120...","[1.1206666666666667, 1.1206666666666667, 1.120...","[1.3565, 1.3565, 1.3564999999999998, 1.3564999...","[1.3560555555555556, 1.3560555555555553, 1.356...","[1.3555000000000001, 1.3555000000000001, 1.355...","[1.5913888888888887, 1.5913888888888887, 1.591...","[1.5909444444444445, 1.5909444444444445, 1.590...","[1.5905000000000002, 1.5905000000000002, 1.590..."
radDeg,"[1.1695034334539784, 1.1618156487621303, 1.154...","[1.125063837320171, 1.1229997919038877, 1.1216...","[1.1292086025850416, 1.1330275364585176, 1.137...","[1.396246438953936, 1.3898194866103237, 1.3839...","[1.3594272273723056, 1.357730027666303, 1.3566...","[1.3626267752007601, 1.365804511052421, 1.3695...","[1.625252467577429, 1.6197474386149016, 1.6147...","[1.593785841040167, 1.5923484180624896, 1.5914...","[1.5966165038801796, 1.5993370545539594, 1.602..."
effarea,"[33.561792066044895, 33.5752324936103, 33.5912...","[33.648290093666446, 33.65117352406033, 33.653...","[33.6424999390284, 33.63716496436353, 33.63086...","[32.92554590494814, 32.94577800379144, 32.9642...","[33.076821191000846, 33.08335645384732, 33.087...","[33.061754206231505, 33.04619508429417, 33.027...","[31.75813063724522, 31.782116587068323, 31.820...","[31.971100407129416, 31.979649419289405, 31.98...","[31.954265160012937, 31.93808480033231, 31.914..."
readnoise,"[6.72664, 6.86398, 6.80134, 6.80359, 6.89268, ...","[6.3264, 6.15465, 6.19607, 6.21983, 6.29398, 6...","[12.4259, 13.4357, 14.2171, 14.4081, 13.8188, ...","[6.34781, 6.38036, 6.47556, 6.41984, 6.57553, ...","[6.35582, 6.4962, 6.66836, 6.64662, 6.66106, 6...","[6.70244, 6.90421, 7.0441, 7.1694, 7.18389, 7....","[8.60859, 8.86458, 8.93421, 8.89687, 8.89008, ...","[6.33231, 6.52467, 6.27177, 6.38031, 6.38624, ...","[6.33007, 6.40012, 6.39609, 6.37229, 6.51182, ..."


In [24]:
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [25]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

u    [24.09667288856489, 24.093538486420435, 24.100...
y    [22.60038470912795, 22.601690090090226, 22.603...
g    [24.902322093826207, 24.90166707051041, 24.906...
r    [24.43155524188687, 24.432513574066157, 24.434...
z    [23.434288547249785, 23.43645875130191, 23.438...
i    [23.998908851675267, 24.00072978552631, 24.002...
Name: R42_S00, dtype: object